# Flow Network

In this notebook we build a network of flows between places, i.e., the number of _trips_ between two areas.

## Preamble

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(context='notebook', font='Lucida Sans Unicode', style='white', palette='plasma')

In [2]:
import pandas as pd
import geopandas as gpd

In [3]:
checkins = pd.read_csv('output/santiago_relevant_check_ins.csv.gz')
checkins.head()

,Unnamed: 0,user_id,venue_id,datetime,utc_offset
0,56,44093,4dc0367c6a23e5a549e68d9b,Tue Apr 03 18:00:22 +0000 2012,-180
1,82,980044,4bb8ddd31261d13ab608e998,Tue Apr 03 18:00:27 +0000 2012,-180
2,83,640995,4f7200c9e4b0e30fde677e23,Tue Apr 03 18:00:28 +0000 2012,-180
3,179,969261,4d386aa93ffba1433b405c56,Tue Apr 03 18:00:55 +0000 2012,-180
4,180,1424666,4b6f2cf8f964a52013e22ce3,Tue Apr 03 18:00:55 +0000 2012,-180


In [4]:
checkins.rename(columns={'Unnamed: 0': 'checkin_id'}, inplace=True)
checkins

,checkin_id,user_id,venue_id,datetime,utc_offset
0,56,44093,4dc0367c6a23e5a549e68d9b,Tue Apr 03 18:00:22 +0000 2012,-180
1,82,980044,4bb8ddd31261d13ab608e998,Tue Apr 03 18:00:27 +0000 2012,-180
2,83,640995,4f7200c9e4b0e30fde677e23,Tue Apr 03 18:00:28 +0000 2012,-180
3,179,969261,4d386aa93ffba1433b405c56,Tue Apr 03 18:00:55 +0000 2012,-180
4,180,1424666,4b6f2cf8f964a52013e22ce3,Tue Apr 03 18:00:55 +0000 2012,-180
...,...,...,...,...,...
1307951,90048214,290903,4f71bccee4b037626f904303,Wed Jan 29 16:41:35 +0000 2014,-180
1307952,90048333,21455,52d6ccb7498ed1693ba6f05d,Wed Jan 29 16:42:18 +0000 2014,-180
1307953,90048359,968028,4b65d266f964a5200b022be3,Wed Jan 29 16:42:29 +0000 2014,-180
1307954,90048399,2194463,4fea56c0e4b09f197e9379c5,Wed Jan 29 16:42:46 +0000 2014,-180


In [5]:
venues = gpd.read_file('output/santiago_relevant_pois.json')
venues.head()

,venue_id,category,index_right,h3_cellid,geometry
0,4af8ffeaf964a520cd1022e3,Other Great Outdoors,2709,88b2c55417fffff,POINT (-70.66046 -33.44616)
1,4b4463faf964a52078f425e3,Sushi Restaurant,1300,88b2c51831fffff,POINT (-70.51793 -33.3738)
2,4b44abe7f964a5201ef925e3,Sushi Restaurant,2980,88b2c556d3fffff,POINT (-70.57765 -33.41222)
3,4b44ad89f964a52040f925e3,Office,1471,88b2c519e7fffff,POINT (-70.57276 -33.40448)
4,4b44b566f964a5200efa25e3,Sandwich Place,2963,88b2c556abfffff,POINT (-70.62263 -33.43098)


In [6]:
venues

,venue_id,category,index_right,h3_cellid,geometry
0,4af8ffeaf964a520cd1022e3,Other Great Outdoors,2709,88b2c55417fffff,POINT (-70.66046 -33.44616)
1,4b4463faf964a52078f425e3,Sushi Restaurant,1300,88b2c51831fffff,POINT (-70.51793 -33.3738)
2,4b44abe7f964a5201ef925e3,Sushi Restaurant,2980,88b2c556d3fffff,POINT (-70.57765 -33.41222)
3,4b44ad89f964a52040f925e3,Office,1471,88b2c519e7fffff,POINT (-70.57276 -33.40448)
4,4b44b566f964a5200efa25e3,Sandwich Place,2963,88b2c556abfffff,POINT (-70.62263 -33.43098)
...,...,...,...,...,...
92226,52e8241811d24e40d89978d0,Home (private),2760,88b2c5549bfffff,POINT (-70.60383 -33.47606)
92227,52e8d7ad498e459461a0b33b,Bus Line,2838,88b2c5555dfffff,POINT (-70.72786 -33.42376)
92228,52e90de1498ed1ed01b47aaf,Bus Line,1954,88b2c5435bfffff,POINT (-70.84306 -33.44869)
92229,52e912e9498ed5442c097af5,Campaign Office,2978,88b2c556cdfffff,POINT (-70.61707 -33.39269)


## Flow Estimation

Check-ins are stored as one row per check-in in the dataframe. However, we need to consider _pairs_ of check-ins by the same user to be able to see a trip.

First, we sort the check-ins dataframe by `user_id`, as that ensures that two consecutive check-ins were done by the same user (with the exception of the last check-in of every user, but we deal with that later).

In [9]:
checkins_grid = checkins.merge(venues[['venue_id', 'index_right', 'h3_cellid']]).sort_values('user_id').drop('utc_offset', axis=1)
checkins_grid.head()

,checkin_id,user_id,venue_id,datetime,index_right,h3_cellid
1158816,73954411,230,4fbd7718e4b06a0de5a8fd1a,Mon Jul 15 20:10:04 +0000 2013,2944,88b2c55681fffff
582498,28707856,230,4b59c564f964a5202c9728e3,Fri Oct 12 12:22:46 +0000 2012,2963,88b2c556abfffff
683061,34870617,230,4b59c564f964a5202c9728e3,Tue Nov 20 14:40:08 +0000 2012,2963,88b2c556abfffff
1108478,69010000,230,4bac9b9af964a520edfd3ae3,Fri May 17 22:51:18 +0000 2013,2238,88b2c546e9fffff
967017,57154026,230,4b59c564f964a5202c9728e3,Fri Mar 22 11:24:37 +0000 2013,2963,88b2c556abfffff


After merging Santiago's check-ins with Santiago's POIs, duplicates must be removed (check-ins must be unique in the table) and sorted by user and then by date, so that the trip can be constructed correctly later on (a trip must be the transfer from one check-in to another check-in in chronological order).

In [10]:
test = checkins_grid.groupby('checkin_id').first().reset_index()
#test = test.sort_values('user_id')
checkins_sorted = test.sort_values(by=['user_id', 'datetime'], ascending=[True, True])
checkins_sorted

,checkin_id,user_id,venue_id,datetime,index_right,h3_cellid
95700,4092006,230,4b549b4ef964a52060c227e3,Fri Apr 20 14:49:25 +0000 2012,2984,88b2c556dbfffff
520911,25283189,230,4c2231a39a67a593ee0ddc87,Fri Aug 10 15:04:41 +0000 2012,483,88b2c50939fffff
777448,41122624,230,4b65adfef964a52047f92ae3,Fri Dec 21 12:13:55 +0000 2012,2723,88b2c55437fffff
777628,41132371,230,5072d1b5e4b0f322440bc73f,Fri Dec 21 12:59:17 +0000 2012,2735,88b2c55453fffff
788255,41962495,230,4b59c564f964a5202c9728e3,Fri Dec 28 15:07:04 +0000 2012,2963,88b2c556abfffff
...,...,...,...,...,...,...
1264120,85318926,2733324,4ba0f706f964a520ad8a37e3,Wed Nov 13 11:14:51 +0000 2013,1450,88b2c519b7fffff
1264616,85364094,2733324,4b82bb3cf964a52007e130e3,Wed Nov 13 16:03:38 +0000 2013,2969,88b2c556b9fffff
1264710,85370915,2733324,4bba03851261d13af117ea98,Wed Nov 13 16:38:23 +0000 2013,1279,88b2c51801fffff
1264869,85381604,2733324,4b901377f964a5201a7433e3,Wed Nov 13 18:05:28 +0000 2013,1457,88b2c519c7fffff


Then, we define an utilitary function `shift` that takes a dataframe as input, and then builds a new dataframe where two consecutive rows of the original dataframe are put together, as if they were from a trip.

Since we have a dataframe sorted by `user_id`, most of the shifted rows belong to only one user. Those that do not are not real trips, and we filter them out of the dataframe.

In [94]:
def shift(df):
    origin = df.rename({'venue_id': 'origin'}, axis=1)[['origin', 'user_id']]
    destination = df.rename({'venue_id': 'destination', 'user_id': 'user_id_d'}, axis=1)[['destination', 'user_id_d']].shift()
    trips = (origin.join(destination)
             .dropna()
             .pipe(lambda x: x[x.user_id == x.user_id_d])
             .groupby(['user_id', 'origin', 'destination'])
             .size()
            )
    trips.name = 'n_trips'
    return trips

#shift(checkins_grid.head(100))

In [12]:
user_trip_counts = shift(checkins_sorted).reset_index()
user_trip_counts.index.name="trip_id"
user_trip_counts.head()

,user_id,origin,destination,n_trips
trip_id,,,,
0,230,4b52419af964a520177327e3,4e3b1d8cfa7645537599360e,1
1,230,4b562eedf964a520180428e3,4b65adfef964a52047f92ae3,1
2,230,4b562eedf964a520180428e3,4d00e897ffcea1434f243091,1
3,230,4b562eedf964a520180428e3,4dff506888779f08f9982716,1
4,230,4b562eedf964a520180428e3,4e78facea809f5bf73502f63,1


In [13]:
user_trip_counts.shape

(990988, 4)

Then, we proceed to assign an origin / destination _cell id_ to each trip.

In [14]:
user_trips_venue_join = (user_trip_counts
.join(venues[['venue_id', 'h3_cellid']].set_index('venue_id'), on='origin')
.rename({'h3_cellid': 'origin_cell_id'}, axis=1))

user_trips_venue = user_trips_venue_join.groupby("trip_id").first().reset_index()

user_trips_venue_join2 = (user_trips_venue.join(venues[['venue_id', 'h3_cellid']].set_index('venue_id'), on='destination')
.rename({'h3_cellid': 'destination_cell_id'}, axis=1))

user_trips_venue = user_trips_venue_join2.groupby("trip_id").first().reset_index()

user_trips_venue

,trip_id,user_id,origin,destination,n_trips,origin_cell_id,destination_cell_id
0,0,230,4b52419af964a520177327e3,4e3b1d8cfa7645537599360e,1,88b2c556b1fffff,88b2c55699fffff
1,1,230,4b562eedf964a520180428e3,4b65adfef964a52047f92ae3,1,88b2c55435fffff,88b2c55437fffff
2,2,230,4b562eedf964a520180428e3,4d00e897ffcea1434f243091,1,88b2c55435fffff,88b2c556a5fffff
3,3,230,4b562eedf964a520180428e3,4dff506888779f08f9982716,1,88b2c55435fffff,88b2c5543bfffff
4,4,230,4b562eedf964a520180428e3,4e78facea809f5bf73502f63,1,88b2c55435fffff,88b2c556b9fffff
...,...,...,...,...,...,...,...
990983,990983,2733324,527b0eae498e3f8e2b08c4c5,4b700ad7f964a52092042de3,1,88b2c51989fffff,88b2c55435fffff
990984,990984,2733324,52864ed8498eab930e507aea,50e582aae4b003353ff24fd4,1,88b2c5094dfffff,88b2c5464dfffff
990985,990985,2733324,52a381a011d2141b9e311b6c,4b6cb3c2f964a5207e4d2ce3,1,88b2c57303fffff,88b2c50907fffff
990986,990986,2733324,52a381a011d2141b9e311b6c,4baa0279f964a5207a433ae3,1,88b2c57303fffff,88b2c51839fffff


Finally, we can compute the flow of people between two cells by aggregating the trips dataframe:

In [15]:
flows = (user_trips_venue
         [user_trips_venue.origin_cell_id != user_trips_venue.destination_cell_id]
         .groupby(['origin_cell_id', 'destination_cell_id'])
         ['n_trips']
         .sum()
         .reset_index()
         .pipe(lambda x: x[x.n_trips > 5])
         .rename({'origin_cell_id': 'origin', 'destination_cell_id': 'dest', 'n_trips': 'count'}, axis=1)
        )
flows.head()

,origin,dest,count
81,88b2c50801fffff,88b2c518adfffff,6
158,88b2c50803fffff,88b2c554e9fffff,10
260,88b2c50815fffff,88b2c50803fffff,9
271,88b2c50821fffff,88b2c50823fffff,8
273,88b2c50821fffff,88b2c50827fffff,16


In [16]:
flows['count'].describe()

count    29180.000000
mean        25.611960
std         66.097165
min          6.000000
25%          8.000000
50%         11.000000
75%         21.000000
max       2249.000000
Name: count, dtype: float64

## Data Export for Visualization

We export the data in the format required in the [flowmap.blue](http://flowmap.blue) platform. It requires a flow magnitude file, and a flow location file.

The flow magnitude was estimated above.

The flow locations are the centroids of each cell. 

In [18]:
grid = gpd.read_file('output/santiago_relevant_grid.geo.json')
grid

,h3_cellid,index_right,geometry
0,88b2c0b259fffff,32,"POLYGON ((-70.95571 -33.64103, -70.96014 -33.6..."
1,88b2c0b60bfffff,33,"POLYGON ((-70.82671 -33.72395, -70.83114 -33.7..."
2,88b2c0b611fffff,33,"POLYGON ((-70.81533 -33.73508, -70.81976 -33.7..."
3,88b2c0b613fffff,33,"POLYGON ((-70.80965 -33.74064, -70.81407 -33.7..."
4,88b2c0b615fffff,33,"POLYGON ((-70.82482 -33.73665, -70.82925 -33.7..."
...,...,...,...
4010,88b2c5ced9fffff,7,"POLYGON ((-70.57109 -33.25692, -70.5755 -33.25..."
4011,88b2c5cedbfffff,7,"POLYGON ((-70.56542 -33.26252, -70.56983 -33.2..."
4012,88b2c5d493fffff,7,"POLYGON ((-70.42577 -33.12758, -70.43017 -33.1..."
4013,88b2c5d497fffff,7,"POLYGON ((-70.43521 -33.12918, -70.43961 -33.1..."


Note that we don't need all cells from the grid, as some of them do not generate or attract trips.

In [28]:
flow_grid = grid[grid.h3_cellid.isin(flows.origin) | grid.h3_cellid.isin(flows.dest)]
flow_grid.shape, grid.shape

((1026, 3), (4015, 3))

In [60]:
centroids = pd.DataFrame({'x': flow_grid.geometry.centroid.x, 'y': flow_grid.geometry.centroid.y, 'cell' : flow_grid.h3_cellid})

C:\Users\sebam\AppData\Local\Temp\ipykernel_15736\3054778301.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = pd.DataFrame({'x': flow_grid.geometry.centroid.x, 'y': flow_grid.geometry.centroid.y, 'cell' : flow_grid.h3_cellid})


In [65]:
flow_locations = centroids[['name', 'y', 'x']].rename({'x': 'lon', 'y': 'lat'}, axis=1)
flow_locations.to_csv('output/santiago_foursquare_flow_locations.csv')

The table with the magnitude of the flows refer to the location of the origin and destination as h3 cells, so it is necessary to indicate the index of the location table.

In [84]:
flow_mag_fix1 = flows.merge(centroids, left_on='origin', right_on='cell', how='left')
flow_mag_fix1['origin'] = flow_mag_fix1['name']
flow_mag_fix1 = flow_mag_fix1.drop(columns=['x', 'y', 'cell', 'name']) 

flow_mag = flow_mag_fix1.merge(centroids, left_on='dest', right_on='cell', how='left')
flow_mag['dest'] = flow_mag['name']
flow_mag = flow_mag.drop(columns=['x', 'y', 'cell', 'name']) 
flow_mag.head()

,origin,dest,count
0,370,1348,6
1,371,2794,10
2,379,371,9
3,384,385,8
4,384,387,16


In [85]:
flow_mag.to_csv('output/santiago_foursquare_flow_magnitudes.csv', index=False)

With these files, we can use flowmap.blue. [See here the resulting visualization!](https://www.flowmap.blue/1QqhqlwA40PzaRMocTy3BNq3djVLc-n6G5Gifwgeh0ew)

## Next Steps

This exploration was only to apply the work of Eduardo Graells in the case of Santiago with foursquare checkins data. The next steps are:

- Inspect EOD data
- Inspect Adatrap data
- Propose a unified format, considering the formats of these datasets
- Make the code that converts the datasets to the unified format.
- Make the code that converts the unified format so that it can be input to flowmap